# 8. Utiliser des APIs

## Lire un fichier
### Lecture de l'intégralité du fichier en une seule chaîne de caractères
```python
with open(path_to_file) as f:
    file_content = f.read()
```

### Lecture du fichier dans une liste de chaines de caractères (1 par ligne)
```python
with open(path_to_file) as f:
    file_lines = f.readlines()
```
Souvent il faut nettoyer les données en supprimant par exemple les sauts de ligne non désirés par exemple.

## requests
### Méthode get
```python
import requests

# URL de base de l'api des utilisateurs dans Alma
API_URL = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/items'
HEADERS = {'content-type': 'application/json',
           'accept': 'application/json',
           'Authorization': 'apikey ' + API_KEY_BIB_SB}
r = requests.get(API_ITEM_URL, params={'item_barcode': 'HR40042121'}, headers=HEADERS)
r.json()
```

### Méthode put
```python
import requests

# URL de base de l'api des utilisateurs dans Alma
API_URL = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/items'
HEADERS = {'content-type': 'application/json',
           'accept': 'application/json',
           'Authorization': 'apikey ' + API_KEY_BIB_SB}
data = json.dumps(json_data)
r = requests.put(API_ITEM_URL, params=params, headers=HEADERS, data=data)
r.json()
```

### Exemple avec du xml
Dans Alma, pour modifier les holdings (et les notices bibliographiques), il est nécessaire de passer par du XML.
```python
from lxml import etree
import requests

API_KEY = 'API_KEY'
HEADERS = {'content-type': 'application/xml',
           'accept': 'application/xml',
           'Authorization': 'apikey ' + API_KEY}
url = f'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}/holdings/{holding_id}'
r1 = requests.get(url,
                 headers=HEADERS)
root = etree.XML(r1.text.encode('utf-8'))
root.find('.//datafield[@tag="852"]/subfield[@code="h"]').text = 'New Callnumber'
data = etree.tostring(root).decode('utf-8')
r2 = requests.put(url,
                 headers=HEADERS,
                 data=data)

root = etree.XML(r2.text.encode('utf-8'))
root.find('.//datafield[@tag="852"]/subfield[@code="h"]').text
```

In [6]:
# Importer les bibliothèques essentielles
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import json
from lxml import etree

PATH_TO_SECRETS = '../Secrets/'

# Afficher toutes les colonnes
pd.set_option('display.max_columns', None)

### Récupération des données d'un exemplaire
Documentation Alma:
* Récupérer les données d'un exemplaire: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/R0VUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0vaXRlbXMve2l0ZW1fcGlkfQ==/
* Modifier un exemplaire: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/UFVUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0vaXRlbXMve2l0ZW1fcGlkfQ==/
* Récupérer les données d'une holding: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/R0VUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0=/
* Modifier une holding: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/UFVUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0=/

In [5]:
# Clé Sandbox en lecture et écriture pour les ressources bibliographiques
with open(PATH_TO_SECRETS + 'API_KEY_ITEMS_SB') as f:
    # Il faut utiliser "strip" pour supprimer le saut de ligne à la fin
    API_KEY_BIB_SB = f.read().strip()

# URL de base de l'api des utilisateurs dans Alma
API_ITEM_URL = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/items'
# En-têtes de la futur requête
# Définit notamment le format de la réponse
HEADERS = {'content-type': 'application/json',
           'accept': 'application/json',
           'Authorization': 'apikey ' + API_KEY_BIB_SB}

In [100]:
df = pd.read_csv('Data/barcode_hph_sb.csv')
df.head()

,Barcode
0,HR40042121
1,02085863
2,03130885
3,02076508
4,1034139759


In [108]:
# Récupération de la cote en fonction du code-barre
r = requests.get(API_ITEM_URL, params={'item_barcode': 'HR40042121'}, headers=HEADERS)

In [109]:
data = r.json()
data

{'bib_data': {'mms_id': '991001605443905520',
  'bib_suppress_from_publishing': 'false',
  'title': 'Oxocard   - code, play and learn unterstützt die freien Blockly-, Python- und C++-Umgebungen :  lerne Animationen und Spiele zu programmieren',
  'author': 'Garaio, Thomas',
  'issn': None,
  'isbn': None,
  'complete_edition': '',
  'network_number': ['(EXLNZ-41SLSP_NETWORK)991170842256805501'],
  'place_of_publication': 'Liebefeld',
  'date_of_publication': '2016-2019',
  'publisher_const': 'Oxon',
  'link': 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/bibs/991001605443905520'},
 'holding_data': {'holding_id': '22105924500005520',
  'holding_suppress_from_publishing': 'false',
  'permanent_call_number_type': {'value': '1',
   'desc': 'Dewey Decimal classification'},
  'permanent_call_number': '681.41//056',
  'call_number_type': {'value': '1', 'desc': 'Dewey Decimal classification'},
  'call_number': '681.41//056',
  'accession_number': '681.41//056',
  'copy_id': '',
  'in_temp

In [110]:
data['item_data']['internal_note_3'] = 'TEST'
url_link = data['link']

# Transformation du json en chaîne de caractères
data = json.dumps(data)
data

'{"bib_data": {"mms_id": "991001605443905520", "bib_suppress_from_publishing": "false", "title": "Oxocard   - code, play and learn unterst\\u00fctzt die freien Blockly-, Python- und C++-Umgebungen :  lerne Animationen und Spiele zu programmieren", "author": "Garaio, Thomas", "issn": null, "isbn": null, "complete_edition": "", "network_number": ["(EXLNZ-41SLSP_NETWORK)991170842256805501"], "place_of_publication": "Liebefeld", "date_of_publication": "2016-2019", "publisher_const": "Oxon", "link": "https://api-eu.hosted.exlibrisgroup.com/almaws/v1/bibs/991001605443905520"}, "holding_data": {"holding_id": "22105924500005520", "holding_suppress_from_publishing": "false", "permanent_call_number_type": {"value": "1", "desc": "Dewey Decimal classification"}, "permanent_call_number": "681.41//056", "call_number_type": {"value": "1", "desc": "Dewey Decimal classification"}, "call_number": "681.41//056", "accession_number": "681.41//056", "copy_id": "", "in_temp_location": false, "temp_library": 

In [111]:
# Modification de l'exemplaire dans Alma
r = requests.put(url_link, data=data, headers=HEADERS)

In [112]:
r.json()

{'bib_data': {'mms_id': '991001605443905520',
  'bib_suppress_from_publishing': 'false',
  'title': 'Oxocard   - code, play and learn unterstützt die freien Blockly-, Python- und C++-Umgebungen :  lerne Animationen und Spiele zu programmieren',
  'author': 'Garaio, Thomas',
  'issn': None,
  'isbn': None,
  'complete_edition': '',
  'network_number': ['(EXLNZ-41SLSP_NETWORK)991170842256805501'],
  'place_of_publication': 'Liebefeld',
  'date_of_publication': '2016-2019',
  'publisher_const': 'Oxon',
  'link': 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/bibs/991001605443905520'},
 'holding_data': {'holding_id': '22105924500005520',
  'holding_suppress_from_publishing': 'false',
  'permanent_call_number_type': {'value': '1',
   'desc': 'Dewey Decimal classification'},
  'permanent_call_number': '681.41//056',
  'call_number_type': {'value': '1', 'desc': 'Dewey Decimal classification'},
  'call_number': '681.41//056',
  'accession_number': '681.41//056',
  'copy_id': '',
  'in_temp

### Récupération des données utilisateurs sur la base du primary id

In [53]:
# Clé NZ en lecture pour les utilisateurs
with open(PATH_TO_SECRETS + 'API_KEY_USERS_NZ') as f:
    # Il faut utiliser "strip" pour supprimer le saut de ligne à la fin
    API_KEY_USERS_NZ = f.read().strip()

# URL de base de l'api des utilisateurs dans Alma
API_BASE_URL = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/users/'

# En-têtes de la futur requête
# Définit notamment le format de la réponse
HEADERS = {'content-type': 'application/json',
           'accept': 'application/json',
           'Authorization': 'apikey ' + API_KEY_USERS_NZ}

In [54]:
primary_id = 'raphael.rey@slsp.ch'

# Effectue la requête
r = requests.get(API_BASE_URL + primary_id, headers=HEADERS)

# Retourne le code de réponse de la requête (moins de 400 c'est bien)
r.status_code

200

In [50]:
# Affiche les données au format json
r.json()

{'record_type': {'value': 'STAFF', 'desc': 'Staff'},
 'primary_id': 'raphael.rey@slsp.ch',
 'first_name': 'Raphaël',
 'middle_name': '',
 'last_name': 'Rey',
 'full_name': 'Raphaël Rey',
 'user_title': {'value': '', 'desc': ''},
 'job_category': {'value': '', 'desc': ''},
 'job_description': '',
 'gender': {'value': '', 'desc': ''},
 'user_group': {'value': '99', 'desc': ' Staff User'},
 'campus_code': {'value': None, 'desc': ''},
 'web_site_url': '',
 'cataloger_level': {'value': '50', 'desc': '[50] GND cataloger / other AUT'},
 'preferred_language': {'value': 'en', 'desc': 'English'},
 'account_type': {'value': 'INTERNAL', 'desc': 'Internal'},
 'external_id': '',
 'password': '',
 'force_password_change': '',
 'status': {'value': 'ACTIVE', 'desc': 'Active'},
 'status_date': '2022-01-07Z',
 'requests': None,
 'loans': None,
 'fees': None,
 'contact_info': {'address': [],
  'email': [{'email_address': 'raphael.rey@slsp.ch',
    'description': None,
    'preferred': True,
    'segment_t

In [9]:
data = r.json()
data['user_group']['desc']

' Staff User'

In [15]:
def get_user_group(data):
    """
    Extract user group from the data.
    """
    try:
        user_group = data['user_group']['desc'].strip()
        
    except KeyError:
        user_group = ''
        
    return user_group

get_user_group(data)


'Staff User'

### Exemple avec du XML

In [44]:
mms_id='991001605443905520'
holding_id = '22105924500005520'

In [45]:
url = f'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}/holdings/{holding_id}'

# Dans le header, il faut préciser que l'on veut du xml
HEADERS = {'content-type': 'application/xml',
           'accept': 'application/xml',
           'Authorization': 'apikey ' + API_KEY_BIB_SB}

r1 = requests.get(url,
                 headers=HEADERS)

# etree ne n'accepte que des bytes. Il faut donc encoder la chaîne
root = etree.XML(r.text.encode('utf-8'))
callnumber = root.find('.//datafield[@tag="852"]')

# etree transfomre l'arbre xml en bytes. Pour en faire une chaîne de caractère, il faut la décoder.
etree.tostring(callnumber).decode('utf-8')

'<datafield ind1="1" ind2=" " tag="852"><subfield code="b">HPHG</subfield><subfield code="c">MI</subfield><subfield code="h">681.41//056</subfield></datafield></record></holding>'

In [46]:
# old_value = '681.41//056'
callnumber.find('.//subfield[@code="h"]').text

'681.41//056'

In [47]:
callnumber.find('.//subfield[@code="h"]').text = '681.41//056'

In [48]:
# Il serait possible d'utiliser `.decode('utf-8')`pour transformer les bytes en string,
# mais ce n'est pas obligatoire avec la biubliothèquze `requests`
data = etree.tostring(root)
r2 = requests.put(url,
                 headers=HEADERS,
                 data=data)

In [49]:
root = etree.XML(r2.text.encode('utf-8'))
root.find('.//datafield[@tag="852"]/subfield[@code="h"]').text

'681.41//056'